# Simple Perceptron Implementation
_by Mihai Dan Nadăș (mihai.nadas@ubbcluj.ro), January 2025_

This notebook implements a version of the perceptron as introduced by Frank Rosenblatt's 1958 paper, "The Perceptron: A Probabilistic Model for Information Storage and Organization in the Brain".

We will focus on basic mathematical concepts and minimize the use of linear algebra, avoiding vector and matrix operations where possible.

## Objective

Our objective is to train a model with two weights, $w_{1}$ and $w_{2}$, corresponding to the coordinates $x$ and $y$ of a point $(x, y)$, plus one bias $b$. This is based on adapting the algebraic equation $y = mx + c$, which represents the slope-intercept form of a line.

The model will address a simple classification task with a linearly separable dataset, generated according to the following function:

Given an integer $x$:

$
f: \mathbb{N} \to \mathbb{N}, \quad f(x) =
\begin{cases}
x, & \text{if } x \bmod 2 = 0, \\
2x, & \text{if } x \bmod 2 = 1.
\end{cases}
$

This function classifies numbers as follows: if $x$ is even, it remains unchanged; if odd, it doubles. This transformation will help classify the numbers based on their parity.

## Dataset
We will generate a dataset using the Python Standard Library.

In [ ]:
import random


def generate_dataset(num_items=20, start=0, stop=100):
    random.seed(42)
    dataset = []
    x1_values = set()
    while len(dataset) < num_items:
        x1 = random.randint(start, stop)
        if x1 in x1_values:
            continue
        x1_values.add(x1)
        x2 = x1 if x1 % 2 == 0 else 2 * x1
        y = (
            0 if x1 == x2 else 1
        )  # (x1, x2) is labeled as Class 0 if x1 is even, and Class 1 otherwise
        dataset.append((x1, x2, y))
    return dataset


dataset = generate_dataset()

# let's now split the dataset into training and test sets
train_ratio = 0.8
num_train = int(len(dataset) * train_ratio)
dataset_train, dataset_test = dataset[:num_train], dataset[num_train:]
print(f"Training set (n={len(dataset_train)}): {dataset_train}")
print(f"Test set (n={len(dataset_test)}: {dataset_test}")

## Visual Representation

In this section, we will visualize the training and test datasets using _Matplotlib_ and _pandas_. This visualization allows us to clearly see the separability of the classes, which is an essential part of understanding how the perceptron algorithm will work. By examining the plots, we can identify the linear decision boundary and assess how well the data can be classified.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


def plot_datasets(train_dataset, test_dataset):
    # Combine datasets into a DataFrame for easier handling
    train_df = pd.DataFrame(train_dataset, columns=["x1", "x2", "class"])
    train_df["set"] = "Train"

    test_df = pd.DataFrame(test_dataset, columns=["x1", "x2", "class"])
    test_df["set"] = "Test"

    combined_df = pd.concat([train_df, test_df], ignore_index=True)

    # Define colors and markers
    colors = {0: "blue", 1: "red"}
    markers = {"Train": "o", "Test": "x"}

    # Plot each group using Matplotlib
    fig, ax = plt.subplots()
    for (dataset, cls), group in combined_df.groupby(["set", "class"]):
        ax.scatter(
            group["x1"],
            group["x2"],
            color=colors[cls],
            label=f"{dataset} Dataset, Class {cls}",
            s=30,
            marker=markers[dataset],
        )

    # Manage legend and labels
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), title="Dataset and Class", loc="best")
    ax.set_xlabel("x1")
    ax.set_ylabel("x2")
    ax.set_title("Training and Test Datasets")
    ax.grid(True)


plot_datasets(dataset_train, dataset_test)
plt.show()

## Defining a Linear Classifier

With our dataset prepared, we now turn to the mathematical foundation that enables our model to classify an input $x_{1}$ and $x_{2}$ as belonging to classes $0$ or $1$. The classification function is defined as follows:

$
c: \mathbb{N} \times \mathbb{N} \to \{0,1\}, \quad
c(x_{1}, x_{2}) =
\begin{cases} 
1, & \text{if } (x_{1}, x_{2}) \in \text{Class 1}, \\
0, & \text{if } (x_{1}, x_{2}) \in \text{Class 0}.
\end{cases}
$

This classification is achieved using the algebraic representation of a line in a Cartesian coordinate system, formulated as:

$
z(x) = w_{1}x_{1} + w_{2}x_{2} + c,
$

where:
- $w_{1}$ and $w_{2}$ are the weights that determine the slope, representing the angle of the line relative to the $x$-axis,
- $c$ is the bias (or intercept), indicating where the line intersects the $y$-axis.

From the plotted graph above, it is evident that the two classes are linearly separable, justifying the use of a linear decision boundary. The weights $w_{1}$ and $w_{2}$ are adjusted through training with Rosenblatt's Perceptron algorithm to achieve this separation effectively.

For better understanding, here's how a line defined by $w_{1} = 1$, $w_{2} = 0.5$, and $c = 0$ would appear on our previous plot, illustrating the separation between the classes.

In [ ]:
zx = lambda x1, x2, w1, w2, c: w1 * x1 + w2 * x2 + c


def plot_zx(w1, w2, c):
    x2 = lambda x1: (
        (-w1 * x1 - c) / w2 if w2 != 0 else -c / w1 if w1 != 0 else c
    )  # this is because the equation of the line is w1*x1 + w2*x2 + c = 0, hence x2 = (-w1*x1 - c) / w2
    x1_values = range(0, 101)
    x2_values = [x2(x1) for x1 in x1_values]
    plt.plot(x1_values, x2_values, label=f"{w1}x1+{w2}x2+{c}=0")
    plt.legend(loc="best")


def plot_datasets_and_zx(w1, w2, c):
    plot_datasets(dataset_train, dataset_test)
    plot_zx(w1, w2, c)


plot_datasets_and_zx(-1.5, 1.1, -10)

Now, it is clear that in this configuration, the datapoints are separated effectively. However, there are other configurations for $w_{1}$, $w_{2}$, and $c$ that result in a less effective example. For instance, when $w_{1} = 0.1$, $w_{2} = 0.1$, and $c = 0.5$, we obtain a less optimal separation boundary.

In [ ]:
plot_datasets_and_zx(0.1, 0.1, 0.5)

In this particular case, the function $z$ is not effective in classifying the data points, indicating that the current weight and bias values are not well-suited for the task.

## Evaluating the Performance of the Classifier

With the decision boundary $z = w_{1}x_{1} + w_{2}x_{2} + c$ defined, our next step is to evaluate its effectiveness using a metric called _accuracy_. This will give us a quantifiable measure of how well our classifier is performing on the dataset.

### Defining the Classifier Function

Before assessing its performance, let's clearly define our classifier. It can be expressed as:

$c: \mathbb{R}^2 \to \{0,1\}, \quad
c(x_{1}, x_{2}) = 
\begin{cases} 
1, & \text{if } z(x_{1}, x_{2}) \geq 0, \\
0, & \text{if } z(x_{1}, x_{2}) < 0.
\end{cases}
$

This means any point $(x_{1}, x_{2})$ that lies above or on the decision boundary, calculated by our linear equation, will be classified as 1. Conversely, points below it will be classified as 0.

Let's implement this classifier in code, and then we'll proceed to discuss and evaluate its performance in more detail.

In [ ]:
cx = lambda x1, x2, w1, w2, c: 1 if zx(x1, x2, w1, w2, c) >= 0 else 0


def accuracy(dataset, w1, w2, c):
    print(f"Calculating accuracy on training set using w1={w1}, w2={w2}, c={c}")
    correct = 0
    for x1, x2, y in dataset:
        if y == cx(x1, x2, w1, w2, c):
            correct += 1
    print(
        f"Resulting accuracy: {correct}/{len(dataset)}, or {correct/len(dataset)*100:.2f}%"
    )
    return correct / len(dataset)


# Applying the accuracy function to the training set using the two sets of weights and bias as shown above, in the first example
accuracy(dataset_train, -1.5, 1.1, -10)

# Applying the accuracy function to the training set using two sets of weights and bias as shown above, in the second example
accuracy(dataset_train, 0.1, 0.1, 0.5)

### Discussion on Accuracy

As shown earlier, changing weights and bias values leads to different accuracy outcomes. This is because the weights and biases define the decision boundary, which affects classification accuracy based on how well it separates the classes in the dataset. Therefore, the challenge is to discover the "optimal" values for these parameters. This is achieved via a process known as _model training_.

## Model Training

We will now train our model using the knowledge from our analysis. This involves a series of iterations to adjust the weights and bias values until a satisfactory level of accuracy is attained. This iterative method helps us find the best combination of parameters tailored to our dataset and classification task. 

### Important Concepts in Model Training

In model training, certain concepts are crucial:

- **Epochs:** This refers to one complete pass through the entire training dataset. Multiple epochs may be required to refine the model.
- **Learning Rate:** This is the step size used when updating the model's parameters. It determines how quickly or slowly the model learns.
- **Weight Updates:** During training, weights are adjusted based on the error of predictions to minimize the divergence from true labels.

These concepts play a pivotal role in training the model effectively to ensure it generalizes well to new data. By iterating with different epochs and learning rates, we can optimize the decision boundary for better classification results.

In [ ]:
# First, let's initialize the weights and bias to zero
w1, w2, c = 0, 0, 0

# Let's now define the learning rate
learning_rate = 0.1

# Let's now define the number of epochs
num_epochs = 100

# Create a DataFrame to store the details of the epochs
epoch_details = pd.DataFrame(columns=["epoch", "x1", "x2", "y", "z", "y_hat", "w1", "w2", "c"])

# Let's now start the training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    for x1, x2, y in dataset_train:
        z = zx(x1, x2, w1, w2, c)
        y_hat = 1 if z >= 0 else 0
        w1 += learning_rate * (y - y_hat) * x1
        w2 += learning_rate * (y - y_hat) * x2
        c += learning_rate * (y - y_hat)
        print(f"  x1={x1}, x2={x2}, y={y}, z={z:.2f}, y_hat={y_hat}, w1={w1:.2f}, w2={w2:.2f}, c={c:.2f}")
        # Append the details to the DataFrame
        epoch_details = epoch_details.concat({
            "epoch": epoch + 1,
            "x1": x1,
            "x2": x2,
            "y": y,
            "z": z,
            "y_hat": y_hat,
            "w1": w1,
            "w2": w2,
            "c": c
        }, ignore_index=True)

# Display the DataFrame
epoch_details.head()